In [ ]:
# pip install qdrant-client groq sentence-transformers dspy-ai fastembed gradio pyjwt --upgrade

In [ ]:
import pandas as pd
df = pd.read_csv("healthcare_dataset.csv")

In [ ]:
def format_row(row):
    return (
        f"Name: {row['Name']}, Age: {row['Age']}, Gender: {row['Gender']}, "
        f"Blood Type: {row['Blood Type']}, Medical Condition: {row['Medical Condition']}, "
        f"Date of Admission: {row['Date of Admission']}, Doctor: {row['Doctor']}, "
        f"Hospital: {row['Hospital']}, Insurance Provider: {row['Insurance Provider']}, "
        f"Billing Amount: {row['Billing Amount']}, Room Number: {row['Room Number']}, "
        f"Admission Type: {row['Admission Type']}, Discharge Date: {row['Discharge Date']}, "
        f"Medication: {row['Medication']}, Test Results: {row['Test Results']}"
        "\n\n".lower()
    )

# Apply the function to each row and create a new column with the formatted text
df['formatted_text'] = df.apply(format_row, axis=1)

# Convert the formatted text into a list (or any other format you need)
text_data = df['formatted_text'].tolist()

In [ ]:
from random import shuffle
sampled_dataset = text_data[:128]
shuffle(sampled_dataset)

In [ ]:
#encoding
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("BAAI/bge-large-en-v1.5", device='cuda')
vectors = model.encode(sampled_dataset)
vectors[0].shape


In [ ]:
import os
os.environ['QDRANT__SERVICE__API_KEY']="qdrant_api_key"
os.environ['QDRANT__SERVICE__JWT_RBAC']='true'

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

# Initialize the client

client = QdrantClient(
    url="https://b3cc6dd2-4f57-4e59-8799-e66f63cccca3.us-east4-0.gcp.cloud.qdrant.io:6333",
    # url='http://localhost:6333',
    api_key=os.environ['QDRANT__SERVICE__API_KEY'],
)

In [ ]:
client.recreate_collection(
    collection_name="phi_data",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

client.upload_collection(
    collection_name="phi_data",
    ids=[i for i in range(len(sampled_dataset))],
    vectors=vectors,
    parallel=4,
    max_retries=3,
)

In [ ]:
def get_context(text):
    query_vector = model.encode(text)

    hits = client.search(
        collection_name="phi_data",
        query_vector=query_vector,
        limit=3  # Return 5 closest points
    )
    s=''
    for x in [sampled_dataset[i.id] for i in hits]:
        s = s + x
    return s

In [ ]:
import jwt
import time


# API key used as the secret to sign the token
api_key = 'eXaMplE12345Key67890Api'


# Current time in seconds since the Unix epoch
current_time = int(time.time())


# JWT payload
payload = {
    'exp': current_time + 3600,  # Token expires in 1 hour
    'value_exists': {
        'collection': 'demo_collection',
        'matches': [
            {'key': 'user', 'value': 'John'}
        ]
    },
    "access": [
    {
        "collection": "demo_collection",
        "access": "r",
        "payload": {
            "user": "John"
      }
    }
  ]  # Read-only global access
}


# Encode the JWT token
encoded_jwt = jwt.encode(payload, api_key, algorithm='HS256')


# Print the JWT token
print(encoded_jwt)

In [ ]:
# create a dummy collection with the original API key

root_client = QdrantClient(
    url="http://localhost:6333",
    api_key="eXaMplE12345Key67890Api",
)

root_client.recreate_collection(
    collection_name="dummy",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

root_client.upload_collection(
    collection_name="dummy",
    ids=[i for i in range(len(sampled_dataset))],
    vectors=vectors,
    parallel=4,
    max_retries=3,
)

In [ ]:

from qdrant_client import QdrantClient, models
import numpy as np

client = QdrantClient(
    url="http://localhost:6333",
    api_key=your_role_key,
)

data = np.array(list([0.1]*1024))
print(data.shape)

client.upload_points(
    collection_name="dummy",
    points=[
        models.PointStruct(
            id="5c56c793-69f3-4fbf-87e6-c4bf54c28c26",
            vector=data,
        )])

In [ ]:
from dspy.retrieve.qdrant_rm import QdrantRM


In [ ]:
qdrant_retriever_model = QdrantRM("phi_data", client, k=3)


In [ ]:
import dspy
llama3 = dspy.GROQ(model='llama3-8b-8192', api_key ="*" )


In [ ]:
dspy.settings.configure(rm=qdrant_retriever_model, lm=llama3)

class GenerateAnswer(dspy.Signature):
    """Answer questions with logical factoid answers."""

    context = dspy.InputField(desc="will contain PHI medical data of patients matched with the query")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="an answer between 10 to 20 words")

In [ ]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = get_context(question)
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)